In [1]:
import numpy as np
import pandas as pd
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re


d:\bigdatatask\kogpt2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = torch.load('model_complete.pth')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
d:\bigdatatask\kogpt2\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\User\AppData\Local\Temp\ipykernel_16428\2116497911.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpicklin

In [5]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "그만":
            break
        a = ""
        print("User > {}".format(q))  # 사용자가 입력한 질문 출력
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + SENT + A_TKN + a)).unsqueeze(dim=0)

            # input_ids를 device로 이동 (GPU로 이동)
            input_ids = input_ids.to(device)

            pred = model(input_ids)
            pred = pred.logits

            # 예측 결과를 CPU로 다시 옮겨서 처리
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]

            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))  # 챗봇 답변 출력


User > 안녕


d:\bigdatatask\kogpt2\Lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Chatbot > 안녕하세요
User > 끝말잇기 알아?
Chatbot > 마음 가는 대로 해보세요
User > 기차
Chatbot > 오늘도 좋은 하루 되세요
User > 아
Chatbot > 네
User > 별로야
Chatbot > 별 거 아니라고 생각해보세요
User > 힘들다 힘들어
Chatbot > 조금만 더 버텨요
User > 장난해?
Chatbot > 재미있나봐요
User > 나 화난다
Chatbot > 제가 다 속상하셨겠어요
User > 뭐라는거야
Chatbot > 저도 궁금해요
User > 싫어
Chatbot > 싫어하지 말아요
User > 싫어
Chatbot > 싫어하지 말아요
User > 싫어
Chatbot > 싫어하지 말아요
User > 싫어
Chatbot > 싫어하지 말아요
User > 너 뭐야
Chatbot > 저는 위로해드리는 로봇이에요
User > 장난하지마
Chatbot > 재미있을 거예요
User > 장난하지마라
Chatbot > 재미있나봐요
